In [8]:
pip install azure-identity

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


This notebooks configures Azure OpenAI Python SDK to use Service Principal in Azure Active Directory and uses Token authentication rather than using Open AI access key. 

This notebook can also run on Databricks with secret scope configured. The Service Principal ID(Client Id), Client Secret and Azure Active Directory Tenant Id can be created in databricks backed vault and retrieved at runtime and stored in environment variables. Azure OpenAI python SDK automatically detects the environment variables and acquires the AAD token. 

The Service Principal needs to be granted "Cognitive Services User" role in Azure portal to the Azure Resource Group which contains the Azure OpenAI Resource or use the below Azure CLI commands.

    openAIRg="<OpenAI Resource Group Name>
    spId="<Service Principal GUID>"
    export resourceId=$(az group show -g $openAIRg | jq -r .id)
    az role assignment create --role "Cognitive Services User" --assignee $spId --scope $resourceId

    Create the Service Principal
    az ad sp create-for-rbac --name myServicePrincipalName --role Cognitive Services User --scopes /subscriptions/Sub ID/resourceGroups/RGName

# Creating Service Principal 

Create the Service Principal

az ad sp create-for-rbac --name myServicePrincipalName --role Cognitive Services User --scopes /subscriptions/Sub ID/resourceGroups/RGName

copy the AppID, App Secret and Tenantid after executing the above command

In [ ]:
import os
#import dbutils

# Specify the name of the secret scope and key vault secret name
#scope_name = "akv"

# Get the secret value
OPENAI_APP_ID = "0fcb761f-8033-4a89-b11f-caf22d77788e"
OPENAI_APP_SECRET = "lr6JoER8EDTFXnFO.16xylUr9wXw7kv7y~"
TENANT_ID = "16b3c013-d300-468d-ac64-7eda0820b6d3"

# Use the secret value in your code
print(OPENAI_APP_ID)
print(OPENAI_APP_SECRET)
print(TENANT_ID)

os.environ["AZURE_TENANT_ID"] = TENANT_ID
os.environ["AZURE_CLIENT_ID"] = OPENAI_APP_ID
os.environ["AZURE_CLIENT_SECRET"] = OPENAI_APP_SECRET

# Option 1: Use DeviceCodeCredential to login interactively using user token

In [18]:
from azure.identity import DeviceCodeCredential
import os

import openai
device_credential = DeviceCodeCredential(tenant_id="XXXXXXXXXXXXXXXXX")
token = device_credential.get_token("https://cognitiveservices.azure.com/.default")

openai.api_type = 'azure_ad'
openai.api_key = token.token

# Option 2: Use DefaultAzureCredential to acquire token using AAD Service Principal





In [20]:
from azure.identity import DefaultAzureCredential
import openai

# Request credential
default_credential = DefaultAzureCredential()
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# Setup parameters
openai.api_type = "azure_ad"
openai.api_key = token.token
openai.api_base = "https://textsearchcurieexercise.openai.azure.com/"
openai.api_version = "2023-03-15-preview"

In [21]:
completion = openai.ChatCompletion.create(deployment_id="chat", messages=[{"role": "user", "content": "Hello world!"}])
print(completion.choices[0].message.content)

Hello there! How can I assist you today?


# Option 3 : Authentication for an (interactive) AAD user

First, give your user the Cognitive Services OpenAI User role in Access Control (IAM) in the Azure Portal. This role allows to use the Azure OpenAI Studio, but does not allow to deploy models and change anything.

In [22]:
import os
import openai
from azure.identity import AzureCliCredential, ChainedTokenCredential, ManagedIdentityCredential, EnvironmentCredential

# Define strategy which potential authentication methods should be tried to gain an access token
credential = ChainedTokenCredential(ManagedIdentityCredential(), EnvironmentCredential(), AzureCliCredential())
access_token = credential.get_token("https://cognitiveservices.azure.com/.default")

# Configure OpenAI SDK to use the access token
openai.api_base = "https://textsearchcurieexercise.openai.azure.com/"
openai.api_version = '2022-12-01'
openai.api_type = 'azure_ad'
openai.api_key = access_token.token
deployment = "davinci"

# Execute completion
prompt = """Write a tagline for an ice cream brand:"""
response = openai.Completion.create(engine=deployment, prompt=prompt, max_tokens=100)
text = response['choices'][0]['text']
print(f"Response was: {text}")


Response was: 

"Taste the Chills of Happiness!"
